In [1]:
# Import necessary libraries
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# Keras libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

2024-09-30 16:13:35.632820: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-30 16:13:35.633168: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-30 16:13:35.635187: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-30 16:13:35.641671: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-30 16:13:35.650412: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been 

In [2]:
# Load the dataset
emoticon_data = pd.read_csv("/home/belief/Desktop/MLProj1/mini-project-1/datasets/train/train_emoticon.csv")
emoticon_data['split_emoticons'] = emoticon_data['input_emoticon'].apply(list)

# Remove the 'input_emoticon' column
emoticon_data = emoticon_data.drop('input_emoticon', axis=1)

In [3]:
# Data encoding
df = emoticon_data
mlb = MultiLabelBinarizer()
emoji_encoded = mlb.fit_transform(df['split_emoticons'])

In [4]:
# Create a DataFrame with the encoded emojis
emoji_df = pd.DataFrame(emoji_encoded, columns=mlb.classes_)

# Concatenate the label column with the encoded emojis
final_df = pd.concat([emoji_df, df['label']], axis=1)

# Separate features (X) and labels (y)
X = final_df.drop('label', axis=1)
y = final_df['label']

In [5]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [6]:
# Convert the labels to categorical values
y_train_cat = tf.keras.utils.to_categorical(y_train, num_classes=len(y.unique()))
y_test_cat = tf.keras.utils.to_categorical(y_test, num_classes=len(y.unique()))

In [7]:
# Build the neural network model using Keras
model = Sequential()
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))  # Input layer and first hidden layer
model.add(Dense(64, activation='relu'))  # Second hidden layer
model.add(Dense(y_train_cat.shape[1], activation='softmax'))  # Output layer

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

/home/belief/Desktop/MLProj1/MLProj1env/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [8]:
# Train the model
model.fit(X_train, y_train_cat, epochs=200, batch_size=32, validation_data=(X_test, y_test_cat))

Epoch 1/200
177/177 ━━━━━━━━━━━━━━━━━━━━ 1s 903us/step - accuracy: 0.4919 - loss: 0.6995 - val_accuracy: 0.4866 - val_loss: 0.6944
Epoch 2/200
177/177 ━━━━━━━━━━━━━━━━━━━━ 0s 469us/step - accuracy: 0.5306 - loss: 0.6906 - val_accuracy: 0.4951 - val_loss: 0.6967
Epoch 3/200
177/177 ━━━━━━━━━━━━━━━━━━━━ 0s 459us/step - accuracy: 0.5625 - loss: 0.6827 - val_accuracy: 0.4936 - val_loss: 0.7023
Epoch 4/200
177/177 ━━━━━━━━━━━━━━━━━━━━ 0s 466us/step - accuracy: 0.5830 - loss: 0.6732 - val_accuracy: 0.4816 - val_loss: 0.7066
Epoch 5/200
177/177 ━━━━━━━━━━━━━━━━━━━━ 0s 460us/step - accuracy: 0.6264 - loss: 0.6556 - val_accuracy: 0.4781 - val_loss: 0.7300
Epoch 6/200
177/177 ━━━━━━━━━━━━━━━━━━━━ 0s 454us/step - accuracy: 0.6559 - loss: 0.6322 - val_accuracy: 0.4605 - val_loss: 0.7658
Epoch 7/200
177/177 ━━━━━━━━━━━━━━━━━━━━ 0s 460us/step - accuracy: 0.7097 - loss: 0.5718 - val_accuracy: 0.4767 - val_loss: 0.8011
Epoch 8/200
177/177 ━━━━━━━━━━━━━━━━━━━━ 0s 516us/step - accuracy: 0.7810 - loss: 0

In [9]:
# Predict on the test data
y_pred_cat = model.predict(X_test)

# Convert predictions back to labels
y_pred = y_pred_cat.argmax(axis=1)

45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 570us/step


In [10]:
# Classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Classification Report:
              precision    recall  f1-score   support

           0       0.44      0.47      0.45       715
           1       0.42      0.40      0.41       701

    accuracy                           0.43      1416
   macro avg       0.43      0.43      0.43      1416
weighted avg       0.43      0.43      0.43      1416

Confusion Matrix:
[[334 381]
 [421 280]]
